In [101]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
nltk.download('stopwords')
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srossgupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # feel free to modify this function if you are storing the API Key differently
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [114]:
def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    # Write solution here
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno<=total):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        print (offsetno)
    return(output)    

In [115]:
api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')

{'error': {'code': 'VALIDATION_ERROR', 'description': 'The supplied limit: -20 is invalid, the maximum is 50.'}}


In [ ]:
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
alldata=[]
ids=[]
name=[]
zips=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        ids.append(each["id"])
        name.append(each["name"])
        zips.append(each["location"]["zip_code"])
        alldata.append(each)

In [73]:
def get_reviews(ids,name):
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    restau_names=[]
    output=[]
    for i in range(len(ids)):
        params = { #parameters are case sensitive!
            "locale": "en_US",
        }
        url = "https://api.yelp.com/v3/businesses/"+str(ids[i])+"/reviews"
        response = requests.get(url,
                            headers=headers,params=params)
        result = response.json()
        output.append(result["reviews"])
        restau_names.append(name[i])
    return output,restau_names
reviews, restau_name= get_reviews(ids,name)

In [84]:
restau=[]
text_reviews=[]
loc_zip=[]
for i in range(len(reviews)):
    for each in output[i]["reviews"]:
        text_reviews.append(each["text"])
#         print(i)
        restau.append(name[i])
        loc_zip.append(zips[i])
d = {'restau_name':restau,'zip':loc_zip,'reviews':text_reviews}
reviews_df = pd.DataFrame(d)

In [86]:
reviews_df.to_csv("reviews_df.csv",index=False)

In [19]:
df = pd.DataFrame(new_data)
df.to_csv("Miami_Yelp.csv",index=False)

In [72]:
df_all = pd.DataFrame(alldata)
df_all.to_csv("Miami_raw.csv",index=False)

-------

In [41]:
df = pd.read_excel("/Users/srossgupta/Desktop/Data Focussed Python/HungryGator/DataFiles/Project_prototype_datafiles.xlsx",sheet_name="MergedData")

In [100]:
new_df = df[df["Source"]=="TripAdvisor"]
new_df["check"] = new_df["Address"].str[-5:].str.isdigit()


/Users/srossgupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [211]:
def check(df):
    try:
        if df["check"]==False:
            df["new_zip"]=""
        if df["check"]==True and df["Address"][-10:].find("-")==-1:
            df["new_zip"]=df["Address"][-5:]
        if df["check"]==False and df["Address"][-10:].find("-")!=-1:
            df["new_zip"]= df["Address"][-10:][:df["Address"][-10:].find("-")]
    except AttributeError: 
        df["new_zip"]=""
    except TypeError:
        df["new_zip"]=""
        
    return df

In [202]:
new= new_df[~new_df.Address.isnull()]

In [209]:
new= new.apply(check, axis=1)
new.to_csv("tripadvisor.csv")

-1
-1
-1
-1
5
rere
-1
5
rere
-1
-1
-1
5
rere
-1
5
rere
-1
5
rere
-1
-1
5
rere
-1
-1
-1
-1
5
rere
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
5
rere
-1
5
rere
-1
5
rere
-1
-1
-1
-1
-1
5
rere
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
5
rere
5
rere
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
5
rere
-1
5
rere
-1
-1
-1
5
rere
-1
-1
5
rere
-1
-1
5
rere
-1
-1
-1
-1
-1
5
rere
5
rere
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
5
rere
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
5
rere
-1
5
rere
5
rere
-1
-1
-1
-1
-1
5
rere
5
rere
5
rere
5
rere
-1
-1
-1
-1
5
rere
5
rere
-1
-1
5
rere
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
5
rere
5
rere
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
-1
-1
-1
-1
-1
5
rere
-1
-1
-1
5
rere

,Restaurant Name,Url,Rating,Votes,Address,Source,Zip,check,new_zip
100,El Malecon Restaurant,http://www.tripadvisor.in/Restaurant_Review-g3...,"1,593 of 3,600",6.0,"5791 SW 137th avenue, Miami, FL",TripAdvisor,"i, FL",False,
101,Met Cafe and Bar,http://www.tripadvisor.in/Restaurant_Review-g3...,"778 of 3,600",10.0,"255 Biscayne Boulevard Way, Hotel Beaux Arts M...",TripAdvisor,33131,True,33131
102,Pollo Tropical,http://www.tripadvisor.in/Restaurant_Review-g3...,NaN,NaN,"8700 SW 40th St, Miami, FL 33165",TripAdvisor,33165,True,33165
103,Oriental Bakery & Grocery Co,http://www.tripadvisor.in/Restaurant_Review-g3...,"990 of 3,600",3.0,"1760 SW 3rd Ave, Miami, FL 33129-1415",TripAdvisor,-1415,False,33129
104,El Nopal,http://www.tripadvisor.in/Restaurant_Review-g3...,NaN,NaN,"22605 S Dixie Hwy, Miami, FL 33170",TripAdvisor,33170,True,33170
105,Iguana Ranas Mexican Grill,http://www.tripadvisor.in/Restaurant_Review-g3...,"849 of 3,600",3.0,"13816 SW 56th St, Miami, FL 33175-6061",TripAdvisor,-6061,False,33175
106,Pasha's,http://www.tripadvisor.in/Restaurant_Review-g3...,"945 of 3,600",1.0,"1120 NW 14th St, Miami, FL 33136",TripAdvisor,33136,True,33136
107,Subway,http://www.tripadvisor.in/Restaurant_Review-g3...,NaN,NaN,"8352 Mills Dr, Miami, FL 33183",TripAdvisor,33183,True,33183
108,Bottega Express,http://www.tripadvisor.in/Restaurant_Review-g3...,NaN,NaN,"111 NW 1st St, Miami, FL 33128",TripAdvisor,33128,True,33128
109,Holy Bagels and Pizzeria of Downtown,http://www.tripadvisor.in/Restaurant_Review-g3...,NaN,NaN,"93 NW 1st St, Miami, FL 33128-1814",TripAdvisor,-1814,False,33128


In [210]:
new.to_csv("tripadvisor.csv")

-----

In [91]:
df_reviews= pd.read_csv("reviews_df.csv")

In [94]:
def datacleaning(df):
    #removing @
    df["reviews"]= df.apply(lambda row: re.sub(r'@[\w0-9]+','',row["reviews"],re.IGNORECASE), axis=1)
    #removing URL tags
    df["reviews"]= df.apply(lambda row: re.sub(r'https?://[\w0-9./]+','',row["reviews"],re.IGNORECASE), axis=1)
    df["reviews"]= df.apply(lambda row: re.sub(r'www.[^ ]+','',row["reviews"],re.IGNORECASE), axis=1)
    
    #removing Byte order marks
    df["reviews"]= df.apply(lambda row: row["reviews"].replace(u"\ufffd", "?"), axis=1)
    #Convert the data to lowercase
    df["reviews"]= df.apply(lambda row: row["reviews"].lower(), axis=1)    
    #removing hashtag/numbers and punctuations
    df["reviews"]= df.apply(lambda row: re.sub(r"[^a-zA-Z ]+",'',row["reviews"]), axis=1)
    return df
df_reviews = datacleaning(df_reviews)

In [97]:
def handle_length(text):
    tokenizer = nltk.tokenize.word_tokenize
    words = tokenizer(text)
    fin=[]
    for word in words:
        if len(word)>1:
            fin.append(word)
    return " ".join(fin)    
df_reviews["reviews"] = df_reviews["reviews"].apply(handle_length)

In [106]:
def filter_data_byzip(df,zip_code):
    df_ = df.copy()
    df_ = df_[df_["zip"]==zip_code]
    return " ".join(list(df_["reviews"]))

In [121]:
val = int(input("Enter the Zipcode:"))
reviews_list= filter_data_byzip(df_reviews,val)
if len(reviews_list)!=0:
    wc = WordCloud(width=1600, height=800,max_font_size=200,stopwords=nltk.corpus.stopwords.words('english')\
                         , colormap="viridis").generate(reviews_list)
    plt.figure(figsize=(20,18))
    plt.imshow(wc, interpolation="bicubic")
    plt.axis("off")
    plt.show()
else:
    print("enter the zip code in the list")

Enter the Zipcode:123
enter the zip code in the list
